In [9]:
import pandas as pd

In [10]:
pd.__version__

'1.4.2'

In [11]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [12]:
dfjan=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
dffeb=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [13]:
dfjan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [14]:
dfjan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [15]:
len(dfjan.columns)

19

In [17]:
dfjan['tpep_pickup_datetime'] = pd.to_datetime(dfjan['tpep_pickup_datetime'])
dfjan['tpep_dropoff_datetime'] = pd.to_datetime(dfjan['tpep_dropoff_datetime'])

dfjan['duration_minutes'] = (dfjan['tpep_dropoff_datetime'] - dfjan['tpep_pickup_datetime']).dt.total_seconds() / 60

std_duration = dfjan['duration_minutes'].std()
std_duration

42.594351241920904

In [19]:
total_records = len(dfjan)

filtered_df = dfjan[(dfjan['duration_minutes'] >= 1) & (dfjan['duration_minutes'] <= 60)]

filtered_records = len(filtered_df)

fraction_left = filtered_records / total_records * 100

print(f"Fraction of records left: {fraction_left:.2f}%")

Fraction of records left: 98.12%


In [21]:
from sklearn.feature_extraction import DictVectorizer
df_subset = dfjan[['PULocationID', 'DOLocationID']].copy()

df_subset['PULocationID'] = df_subset['PULocationID'].astype(str)
df_subset['DOLocationID'] = df_subset['DOLocationID'].astype(str)

train_dicts = df_subset.to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print("Feature matrix shape:", X_train.shape)
print("Number of features (columns):", X_train.shape[1])



Feature matrix shape: (3066766, 518)
Number of features (columns): 518


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
y_train = dfjan['duration_minutes'].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"RMSE on training data: {rmse:.2f}")

RMSE on training data: 42.00


In [1]:
dffeb['duration_minutes'] = (dffeb['tpep_dropoff_datetime'] - dffeb['tpep_pickup_datetime']).dt.total_seconds() / 60
val_dicts = dffeb[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = dffeb['duration_minutes'].values

y_pred = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"Validation RMSE: {rmse_val:.2f}")

NameError: name 'dffeb' is not defined